In [ ]:
!pip install pykan

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install torch
import torch
from kan import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'torch._prims_common'

In [ ]:
import pandas as pd
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
data = pd.read_excel("data_1405.xlsx")

In [ ]:
data.head()

In [ ]:
data = data.dropna()

In [ ]:
X = data.drop(columns=['TYPE'])
y = data['TYPE']

In [ ]:
X_array = X.to_numpy()

In [ ]:
    # Convert to PyTorch tensors
data_tensor = torch.tensor(X_array, dtype=torch.float32)
target_tensor = torch.tensor(y, dtype=torch.long)

In [ ]:
 train_data, test_data, train_target, test_target = train_test_split(data_tensor, target_tensor, test_size=0.2, random_state=42)

In [ ]:
# Create data loaders (optional, if you want to batch and shuffle the data)
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_target), batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_target), batch_size=1, shuffle=False)

In [ ]:
train_inputs = torch.empty(0, 8, device=device)
train_labels = torch.empty(0, dtype=torch.long, device=device)
test_inputs = torch.empty(0, 8, device=device)
test_labels = torch.empty(0, dtype=torch.long, device=device)

In [ ]:
 # Concatenate all data into a single tensor on the specified device
for data, labels in train_loader:
      train_inputs = torch.cat((train_inputs, data.to(device)), dim=0)
      train_labels = torch.cat((train_labels, labels.to(device)), dim=0)

for data, labels in test_loader:
      test_inputs = torch.cat((test_inputs, data.to(device)), dim=0)
      test_labels = torch.cat((test_labels, labels.to(device)), dim=0)

dataset = {}
dataset['train_input'] = train_inputs
dataset['test_input'] = test_inputs
dataset['train_label'] = train_labels
dataset['test_label'] = test_labels



In [ ]:
print("Train data shape: {}".format(dataset['train_input'].shape))
print("Train target shape: {}".format(dataset['train_label'].shape))
print("Test data shape: {}".format(dataset['test_input'].shape))
print("Test target shape: {}".format(dataset['test_label'].shape))

In [ ]:
model = KAN(width=[8, 9, 6], grid=5, k=6, seed=0, device=device)

In [ ]:
model(dataset['train_input'])

In [ ]:
model.plot(beta=100, scale=1, in_vars=['BYTES',	'BYTES_REV',	'PACKETS','PACKETS_REV'	,'DBI_BRST_BYTES_MEAN',	'DBI_BRST_PACKETS_MEAN',	'PKT_LENGTHS_MEAN'	,'BRST_DURATION_MEAN'], out_vars=["0","1","2","3","4","5"])

In [ ]:
def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).float())

results = model.train(dataset, opt="Adam", device=device, metrics=(train_acc, test_acc),
                      loss_fn=torch.nn.CrossEntropyLoss(), steps=100, lamb=0.01, lamb_entropy=10., save_fig=True)


In [ ]:
results['train_acc'][-1], results['test_acc'][-1]